<a href="https://colab.research.google.com/github/dansarmiento/analytics_portfolio/blob/main/dbt_bigquery_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook provides a practical, hands-on demonstration of how to implement a complete dbt-core analytics engineering workflow within a Google Colab environment. It leverages the power of Google BigQuery as a serverless cloud data warehouse and dbt-core for robust, version-controlled data transformation. The project serves as a comprehensive guide for building and testing a production-style data pipeline from raw data ingestion to final model auditing, all within a free, cloud-based platform.

**Executive Summary**

This project successfully built and executed an end-to-end data transformation pipeline. The key accomplishments are as follows:

- Data Ingestion: Raw customer, order, and payment data was extracted from a public Google Sheet and successfully loaded into dedicated tables within a jaffle_shop_raw dataset in Google BigQuery.
- dbt Transformation: A dbt project was configured from scratch to connect to the BigQuery data warehouse. A complex SQL model was developed to join and transform the raw tables into a final, enriched customers table for analysis.
- Successful Execution: The dbt run command successfully executed the transformation pipeline, materializing the final model as a new, production-ready table in a jaffle_shop_dbt dataset.
- Data Quality & Auditing: To validate the model, the dbt-labs/audit_helper package was installed and used to perform a data audit, successfully verifying that the model's output was consistent and correct.
- Outcome: The notebook demonstrates a complete and successful ELT (Extract, Load, Transform) process using modern, in-demand data tools, showcasing key skills in data modeling, cloud data warehousing, and data quality assurance.

# Phase 1: Setup and Authentication
**Rationale**

Before using dbt with a cloud data warehouse like BigQuery, the Colab environment must be securely connected to a specific Google Cloud Project. This phase handles the one-time authentication and API activation necessary for all subsequent steps.

**Commands & Explanations**
- auth.authenticate_user(): This command triggers the standard Google OAuth pop-up window. It allows you, the user, to grant the temporary Colab session permission to act on your behalf and use Google Cloud services.
- !gcloud projects list: Lists all Google Cloud projects your authenticated account has access to. This is used to find the correct PROJECT_ID for configuration.
- !gcloud config set project <PROJECT_ID>: This command sets the default project for the gcloud command-line tool within the Colab session. It resolves ambiguity and ensures subsequent commands target the correct project.
- !gcloud services enable bigquery.googleapis.com: This is a one-time activation for your project. It "turns on" the BigQuery service, allowing your project to accept and process API requests.
- !pip install dbt-bigquery pandas-gbq: This installs the necessary Python packages:
- dbt-bigquery: The dbt adapter that allows dbt to understand and communicate with BigQuery.
- pandas-gbq: A library that enables pandas DataFrames to be easily uploaded to BigQuery tables.

In [1]:
# Authenticate your Google user to access Google Cloud services
from google.colab import auth
auth.authenticate_user()

print('Authenticated Successfully.')

Authenticated Successfully.


In [2]:
!gcloud projects list

PROJECT_ID                NAME              PROJECT_NUMBER
ancient-sunspot-332819    My First Project  1058235368387
mineral-service-231322    google sheets     539203237789
quickstart-1549828717454  Quickstart        242952853975


In [3]:
# Gcloud config setup
!gcloud config set project 1058235368387

Updated property [core/project].


In [4]:
# Get the project ID
project_id = !gcloud config list project --format "value(core.project)"
project_id = project_id[0]
print(f"Using Project ID: {project_id}")

# Enable the BigQuery API
print("\nEnabling the BigQuery API...")
!gcloud services enable bigquery.googleapis.com --project {project_id}
print("BigQuery API enabled.")

Using Project ID: 1058235368387

Enabling the BigQuery API...
BigQuery API enabled.


In [5]:
!pip install dbt-bigquery pandas-gbq -q

print("Required packages installed.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.4/114.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 946.9/946.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.9/144.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.7/442.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.

# Phase 2: Loading Raw Data into BigQuery
**Rationale**

dbt is a tool for Transformation, the "T" in ELT (Extract, Load, Transform). This phase performs the "E" and "L": extracting data from the source (Google Sheets) and loading it into a raw dataset in our data warehouse (BigQuery). This gives dbt the raw material it needs to work with.

**Commands & Explanations**

- pd.read_csv(gsheet_url): This line uses the pandas library to read data directly from a Google Sheet that has been published as a CSV.
- clean_bq_column_names(df): A helper function you created to automatically sanitize the column headers from the Google Sheet. It replaces spaces and special characters (like ? and /) with underscores (_) to make the column names valid for BigQuery.
- df.to_gbq(...): The key function that uploads the cleaned pandas DataFrame into a BigQuery table.
- destination_table: The full name of the target table in BigQuery, formatted as dataset_name.table_name.
- project_id: Specifies the target Google Cloud project where the table will be created.
- if_exists='replace': A convenience setting that tells BigQuery to overwrite the table if it already exists, making the script re-runnable without errors.

In [6]:
import pandas as pd
import pandas_gbq
import re

# Loading data from google sheets
url = 'https://docs.google.com/spreadsheets/d/1z8iP62Ip5cZDiHvX1lBmxLITii6cEsBzdtnb1_KDiog/edit?usp=sharing'
match = re.search(r'/d/([a-zA-Z0-9-_]+)', url)
if not match:
    raise ValueError("Invalid Google Sheets URL")
sheet_id = match.group(1)
sheet_names = ['orders', 'customers', 'stripe_payments']

# The name for our new BigQuery dataset for raw data
raw_dataset_name = 'jaffle_shop_raw'

print(f"Fetching data from Google Sheets and loading into BigQuery Project '{project_id}'...")

for sheet_name in sheet_names:
    try:
        gsheet_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
        df = pd.read_csv(gsheet_url)

        # Define the full table ID for BigQuery
        table_id = f'{raw_dataset_name}.{sheet_name}'

        pandas_gbq.to_gbq(
            df,
            destination_table=table_id,
            project_id=project_id,
            if_exists='replace'
        )

        print(f"  Successfully loaded '{sheet_name}' sheet to BigQuery table '{table_id}'")

    except Exception as e:
        print(f"  Could not load sheet '{sheet_name}'. Error: {e}")

Fetching data from Google Sheets and loading into BigQuery Project '1058235368387'...


100%|██████████| 1/1 [00:00<00:00, 7738.57it/s]


  Successfully loaded 'orders' sheet to BigQuery table 'jaffle_shop_raw.orders'


100%|██████████| 1/1 [00:00<00:00, 2623.08it/s]


  Successfully loaded 'customers' sheet to BigQuery table 'jaffle_shop_raw.customers'


100%|██████████| 1/1 [00:00<00:00, 2608.40it/s]

  Successfully loaded 'stripe_payments' sheet to BigQuery table 'jaffle_shop_raw.stripe_payments'


In [7]:
table_names = ['orders', 'customers', 'stripe_payments']

print(f"--- Verifying Raw Data in BigQuery Dataset: {raw_dataset_name} ---\n")

for table in table_names:
    try:
        sql = f"SELECT * FROM `{project_id}.{raw_dataset_name}.{table}` LIMIT 5"

        # Use pandas_gbq to run a query and load results into a DataFrame
        df_sample = pandas_gbq.read_gbq(sql, project_id=project_id)

        print(f"Sample from raw table: {raw_dataset_name}.{table}")
        display(df_sample)
        print("-" * 70)

    except Exception as e:
        print(f"Could not query table {raw_dataset_name}.{table}. Error: {e}")

--- Verifying Raw Data in BigQuery Dataset: jaffle_shop_raw ---

Downloading: 100%|██████████|
Sample from raw table: jaffle_shop_raw.orders


,ID,USER_ID,ORDER_DATE,STATUS
0,2,3,1/2/2018,completed
1,3,94,1/4/2018,completed
2,4,50,1/5/2018,completed
3,5,64,1/5/2018,completed
4,6,54,1/7/2018,completed


----------------------------------------------------------------------
Downloading: 100%|██████████|
Sample from raw table: jaffle_shop_raw.customers


,ID,FIRST_NAME,LAST_NAME
0,20,Anna,A.
1,23,Mildred,A.
2,40,Maria,A.
3,59,Adam,A.
4,74,Harry,A.


----------------------------------------------------------------------
Downloading: 100%|██████████|
Sample from raw table: jaffle_shop_raw.stripe_payments


,ID,ORDERID,PAYMENTMETHOD,STATUS,AMOUNT,CREATED
0,11,9,bank_transfer,success,0,1/12/2018
1,80,65,credit_card,success,0,3/8/2018
2,93,77,credit_card,success,0,3/21/2018
3,3,3,coupon,success,100,1/4/2018
4,14,12,credit_card,success,100,1/15/2018


----------------------------------------------------------------------


# Phase 3: dbt Project Configuration
**Rationale**

This phase involves creating the core configuration files that define a dbt project. These files tell dbt how to operate: how to connect to the database, where your SQL models are located, and how they should be materialized.

**File: profiles.yml**
- Purpose: This file configures the database connection. It is intentionally kept separate from the project files so that sensitive credentials are not stored in version control.
- Fields:
  - my_bq_project: A custom name for this connection profile.
  - target: dev: Specifies that the default connection settings to use are those under the dev key.
  - type: bigquery: Tells dbt to use the dbt-bigquery adapter.
  - method: oauth: Instructs dbt to use the credentials generated by Colab's auth.authenticate_user() flow.
  - project: Your specific Google Cloud Project ID. This is where dbt will run jobs.
  - dataset: The BigQuery dataset where dbt will build its transformed tables and views.
**File: dbt_project.yml**
  - Purpose: This is the main configuration file for this specific dbt project.
  - Fields:
    - name: The name of your dbt project.
    - profile: This critical key tells dbt which profile to use from your profiles.yml file, linking the project to the database connection.
    - model-paths: An array of directories where dbt will look for your .sql model files.
    - +materialized: table: A project-wide configuration that sets the default materialization for all models in the marts sub-directory to table. This means dbt will create physical tables instead of views

In [8]:
# Create the main project folder and subdirectories for models and analysis
!mkdir -p my_bigquery_project/models/marts
!mkdir -p my_bigquery_project/analysis

# Create the dbt configuration directory
!mkdir -p /root/.dbt/

print("dbt project directories created.")

dbt project directories created.


In [9]:
%%writefile /root/.dbt/profiles.yml
my_bq_project:
  target: dev
  outputs:
    dev:
      type: bigquery
      method: oauth
      project: "1058235368387" # <-- PASTE YOUR PROJECT ID HERE
      dataset: jaffle_shop_dbt # This is where dbt will build its models
      threads: 4
      location: US # Or your preferred BigQuery location

Writing /root/.dbt/profiles.yml


In [10]:
%%writefile my_bigquery_project/dbt_project.yml
name: 'my_bigquery_project'
version: '1.0.0'
profile: 'my_bq_project' # This must match the name in profiles.yml

model-paths: ["models"]
analysis-paths: ["analysis"]

models:
  my_bigquery_project:
    marts:
      +materialized: table # Models in the 'marts' folder will be created as tables

Writing my_bigquery_project/dbt_project.yml


# Phase 4: dbt Modeling
**Rationale**

This is the core of dbt's purpose. Here, we formally declare our raw data sources and write SQL SELECT statements (called "models") that transform the raw data into clean, reliable, and analysis-ready tables.

**File: models/sources.yml**
- Purpose: This file declares and documents the raw data tables that the dbt project will use as its starting points. This allows you to test your assumptions about the source data.
- Fields:
  - database: The GCP project where the raw data lives.
  - schema: The BigQuery dataset where the raw tables are located (e.g., jaffle_shop_raw).
  - tables: A list of the source tables.
  - tests: A block where you can define source data tests. The relationships test shown here ensures that every orderid in the stripe_payments table has a matching id in the orders table, enforcing referential integrity.
**File: models/marts/customers.sql**
- Purpose: This is the primary transformation logic. It is a single .sql file containing one SELECT statement that joins data from multiple sources to produce a final, valuable table.
- Syntax:
    - {{ source('jaffle_shop', 'orders') }}: This is a call to a dbt Jinja function. It references the sources you defined in sources.yml and gets replaced with the full table name (e.g., 1058235368387.jaffle_shop_raw.orders) at runtime.
    - WITH ... AS (...): Standard SQL Common Table Expressions (CTEs) are used heavily to break down complex logic into sequential, readable steps.

In [11]:
%%writefile my_bigquery_project/models/sources.yml
version: 2

sources:
  - name: jaffle_shop # A logical name for the source
    database: "1058235368387" # <-- PASTE YOUR PROJECT ID HERE
    schema: jaffle_shop_raw # The dataset containing our raw tables
    tables:
      - name: customers
      - name: orders
      - name: stripe_payments
        # Renaming columns from the raw source is a best practice
        columns:
          - name: id
            description: The primary key for payments.
          - name: orderid
            tests:
              - relationships:
                  to: source('jaffle_shop', 'orders')
                  field: id

Writing my_bigquery_project/models/sources.yml


In [12]:
%%writefile my_bigquery_project/models/marts/customers.sql
WITH paid_orders as (
    select
        orders.id as order_id,
        orders.user_id as customer_id,
        orders.order_date as order_placed_at,
        orders.status as order_status,
        p.total_amount_paid,
        p.payment_finalized_date,
        c.first_name as customer_first_name,
        c.last_name as customer_last_name
    FROM
        {{ source('jaffle_shop', 'orders') }} as orders
    left join
        (
            select
                orderid as order_id,
                max(created) as payment_finalized_date,
                sum(amount) / 100.0 as total_amount_paid
            from
                {{ source('jaffle_shop', 'stripe_payments') }}
            where
                status <> 'fail'
            group by
                1
        ) p ON orders.id = p.order_id
    left join
        {{ source('jaffle_shop', 'customers') }} as c on orders.user_id = c.id
),

customer_orders as (
    select
        c.id as customer_id,
        min(order_date) as first_order_date,
        max(order_date) as most_recent_order_date,
        count(orders.id) AS number_of_orders
    from
        {{ source('jaffle_shop', 'customers') }} as c
    left join
        {{ source('jaffle_shop', 'orders') }} as orders on orders.user_id = c.id
    group by
        1
)

select
    p.*,
    ROW_NUMBER() OVER (ORDER BY p.order_id) as transaction_seq,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY p.order_id) as customer_sales_seq,
    CASE
        WHEN c.first_order_date = p.order_placed_at THEN 'new'
        ELSE 'return'
    END as nvsr,
    x.clv_bad as customer_lifetime_value,
    c.first_order_date as fdos
FROM
    paid_orders p
left join
    customer_orders as c USING (customer_id)
LEFT OUTER JOIN
    (
        select
            p.order_id,
            sum(t2.total_amount_paid) as clv_bad
        from
            paid_orders p
        left join
            paid_orders t2 on p.customer_id = t2.customer_id and p.order_id >= t2.order_id
        group by
            1
        order by
            p.order_id
    ) x on x.order_id = p.order_id
ORDER BY
    order_id

Writing my_bigquery_project/models/marts/customers.sql


# Phase 5: dbt Execution and Auditing
**Rationale**

After defining the models, this phase executes the transformations and then performs an advanced auditing step. This mimics a real-world workflow where an engineer refactors a model and then verifies that the new version produces identical results to the old one.

**Commands & Explanations**
- !dbt run: The primary command to execute your dbt project. It finds all models, compiles the Jinja into pure SQL, and runs that SQL against BigQuery to create your final tables.
- packages.yml: A file used to declare external dbt packages (libraries of reusable macros) that you want to include in your project.
- !dbt deps: The command that reads packages.yml and installs the specified packages into the dbt_packages folder in your project.
- !cp ...: A simple shell command used to create a "legacy" copy of the customers model, simulating a before-and-after scenario for our audit.
- analysis/compare_models.sql: Analysis files are for queries you want dbt to compile but not run during a normal dbt run. They are perfect for one-off checks and audits.
- {{ audit_helper.compare_row_counts(...) }}: A call to a macro from the imported audit_helper package. This specific macro generates a complete SQL query that counts the rows in two different models and presents them for comparison.
- !dbt show --select compare_models: This command compiles and runs a specific analysis file and displays the results directly to the console without creating a permanent table in the warehouse. It is the command-line equivalent of clicking a "Preview" button in dbt Cloud.

In [13]:
%cd my_bigquery_project
!dbt run

/content/my_bigquery_project
20:52:07  Running with dbt=1.9.6
20:52:22  Registered adapter: bigquery=1.9.2
20:52:22  Unable to do partial parsing because saved manifest not found. Starting full parse.
20:52:24  Found 1 model, 1 test, 3 sources, 493 macros
20:52:24  
20:52:24  Concurrency: 4 threads (target='dev')
20:52:24  
20:52:24  1 of 1 START sql table model jaffle_shop_dbt.customers ......................... [RUN]
20:52:27  1 of 1 OK created sql table model jaffle_shop_dbt.customers .................... [CREATE TABLE (99.0 rows, 9.7 KiB processed) in 2.89s]
20:52:27  
20:52:27  Finished running 1 table model in 0 hours 0 minutes and 3.38 seconds (3.38s).
20:52:27  
20:52:27  Completed successfully
20:52:27  
20:52:27  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 TOTAL=1


In [14]:
%%writefile /content/my_bigquery_project/packages.yml
packages:
  - package: dbt-labs/audit_helper
    version: [">=0.9.0", "<1.0.0"]

Writing /content/my_bigquery_project/packages.yml


In [15]:
%cd /content/my_bigquery_project
!dbt deps

/content/my_bigquery_project
20:52:31  Running with dbt=1.9.6
20:52:31  Updating lock file in file path: /content/my_bigquery_project/package-lock.yml
20:52:31  Installing dbt-labs/audit_helper
20:52:31  Installed from version 0.12.1
20:52:31  Up to date!
20:52:31  Installing dbt-labs/dbt_utils
20:52:32  Installed from version 1.3.0
20:52:32  Up to date!


In [16]:
# Create a "legacy" copy of the model for comparison
!cp /content/my_bigquery_project/models/marts/customers.sql /content/my_bigquery_project/models/marts/customers_legacy.sql

print("Created 'customers_legacy.sql' for comparison.")

Created 'customers_legacy.sql' for comparison.


In [17]:
%%writefile /content/my_bigquery_project/analysis/compare_models.sql
{{ audit_helper.compare_row_counts(
    a_relation=ref('customers_legacy'),
    b_relation=ref('customers')
) }}

Writing /content/my_bigquery_project/analysis/compare_models.sql


In [18]:
%cd /content/my_bigquery_project
!dbt run

/content/my_bigquery_project
20:52:34  Running with dbt=1.9.6
20:52:38  Registered adapter: bigquery=1.9.2
20:52:39  Unable to do partial parsing because a project dependency has been added
20:52:42  Found 2 models, 1 analysis, 1 test, 3 sources, 658 macros
20:52:42  
20:52:42  Concurrency: 4 threads (target='dev')
20:52:42  
20:52:42  1 of 2 START sql table model jaffle_shop_dbt.customers ......................... [RUN]
20:52:42  2 of 2 START sql table model jaffle_shop_dbt.customers_legacy .................. [RUN]
20:52:45  1 of 2 OK created sql table model jaffle_shop_dbt.customers .................... [CREATE TABLE (99.0 rows, 9.7 KiB processed) in 3.08s]
20:52:46  2 of 2 OK created sql table model jaffle_shop_dbt.customers_legacy ............. [CREATE TABLE (99.0 rows, 9.7 KiB processed) in 3.12s]
20:52:46  
20:52:46  Finished running 2 table models in 0 hours 0 minutes and 3.59 seconds (3.59s).
20:52:46  
20:52:46  Completed successfully
20:52:46  
20:52:46  Done. PASS=2 WARN=0 E

In [19]:
final_model_dataset = 'jaffle_shop_dbt' # The dataset you defined in profiles.yml
final_model_name = 'customers'          # The name of your .sql model file

# The fully-qualified table name for your dbt model
model_table_id = f"`{project_id}.{final_model_dataset}.{final_model_name}`"
sql = f"SELECT * FROM {model_table_id} LIMIT 10"

print(f"--- Previewing Transformed dbt Model: {final_model_dataset}.{final_model_name} ---\n")

try:
    df_final = pandas_gbq.read_gbq(sql, project_id=project_id)
    display(df_final)
except Exception as e:
    print(f"Could not query the dbt model table. Make sure 'dbt run' completed successfully.")
    print(f"Error: {e}")

--- Previewing Transformed dbt Model: jaffle_shop_dbt.customers ---

Downloading: 100%|██████████|


,order_id,customer_id,order_placed_at,order_status,total_amount_paid,payment_finalized_date,customer_first_name,customer_last_name,transaction_seq,customer_sales_seq,nvsr,customer_lifetime_value,fdos
0,1,1,1/1/2018,returned,10.0,1/1/2018,Michael,P.,1,1,new,10.0,1/1/2018
1,2,3,1/2/2018,completed,20.0,1/2/2018,Kathleen,P.,2,1,new,20.0,1/2/2018
2,3,94,1/4/2018,completed,1.0,1/4/2018,Gregory,H.,3,1,return,1.0,1/29/2018
3,4,50,1/5/2018,completed,25.0,1/5/2018,Billy,L.,4,1,new,25.0,1/5/2018
4,5,64,1/5/2018,completed,17.0,1/5/2018,David,C.,5,1,return,17.0,1/20/2018
5,6,54,1/7/2018,completed,6.0,1/7/2018,Rose,M.,6,1,return,6.0,1/22/2018
6,7,88,1/9/2018,completed,16.0,1/9/2018,Adam,T.,7,1,new,16.0,1/9/2018
7,8,2,1/11/2018,returned,23.0,1/11/2018,Shawn,M.,8,1,new,23.0,1/11/2018
8,9,53,1/12/2018,completed,23.0,1/12/2018,Anne,B.,9,1,new,23.0,1/12/2018
9,10,7,1/14/2018,completed,26.0,1/14/2018,Martin,M.,10,1,new,26.0,1/14/2018


In [20]:
%cd /content/my_bigquery_project
!dbt show --select compare_models

/content/my_bigquery_project
20:52:50  Running with dbt=1.9.6
20:52:54  Registered adapter: bigquery=1.9.2
20:52:55  Found 2 models, 1 analysis, 1 test, 3 sources, 658 macros
20:52:55  
20:52:55  Concurrency: 4 threads (target='dev')
20:52:55  
Previewing node 'compare_models':
| relation_name        | total_records |
| -------------------- | ------------- |
| `1058235368387`.`... |            99 |
| `1058235368387`.`... |            99 |



# Phase 6: Implement Staging and Sources
**Rationale:**

Our current customers model is a monolithic query that references raw data directly. While functional, this is not a scalable or maintainable practice. We will now refactor our project to introduce a staging layer, a core concept in dbt.

A staging model has a 1-to-1 relationship with a raw source table. Its only purpose is to perform light, essential transformations:

- Renaming columns for clarity (e.g., id to customer_id).
- Casting data types.
- Performing basic calculations (e.g., converting cents to dollars).

This creates a clean, consistent, and documented foundation. All downstream models will then select from these staging models using ref(), never touching the raw data directly. This improves data lineage, simplifies debugging, and makes the entire project more modular and readable.

**Create Staging Models**

Now, we create one new .sql file for each of our three raw tables. These models simply select and rename columns from the source, creating a clean foundation for our final model. By default, these will be materialized as views, which is efficient for simple transformations.

In [21]:
# Define the path for the main staging directory
staging_dir = "/content/my_bigquery_project/models/staging"

# Create the staging directory and all necessary subfolders
!mkdir -p {staging_dir}/jaffle_shop
!mkdir -p {staging_dir}/stripe
print("Staging directories created successfully.")



Staging directories created successfully.


In [22]:
%%writefile {staging_dir}/sources.yml
version: 2

sources:
  - name: jaffle_shop # Source system 1
    # This inherits the project from your profiles.yml
    schema: jaffle_shop_raw # The dataset with raw tables
    tables:
      - name: customers
      - name: orders

  - name: stripe # Source system 2
    schema: jaffle_shop_raw
    tables:
      - name: stripe_payments

Writing /content/my_bigquery_project/models/staging/sources.yml


In [23]:
%%writefile /content/my_bigquery_project/models/staging/jaffle_shop/stg_customers.sql

select
    id as customer_id,
    name as full_name,
    -- All other columns from the source
    * except (id, name)

from {{ source('jaffle_shop', 'customers') }}

Writing /content/my_bigquery_project/models/staging/jaffle_shop/stg_customers.sql


In [24]:
%%writefile /content/my_bigquery_project/models/staging/jaffle_shop/stg_orders.sql

select
    id as order_id,
    user_id as customer_id,
    order_date,
    status as order_status

from {{ source('jaffle_shop', 'orders') }}

Writing /content/my_bigquery_project/models/staging/jaffle_shop/stg_orders.sql


In [25]:
%%writefile /content/my_bigquery_project/models/staging/stripe/stg_payments.sql

select
    id as payment_id,
    orderid as order_id,
    paymentmethod,
    status as payment_status,
    -- Convert amount from cents to dollars
    amount / 100 as amount,
    created as created_at

from {{ source('stripe', 'stripe_payments') }}

Writing /content/my_bigquery_project/models/staging/stripe/stg_payments.sql


# Refactor Your Final Model
Now, we update your main customers.sql model. Instead of referencing raw source data, it will now reference your clean staging models using the ref() function. This makes the code much more readable and modular.

In [26]:
%%writefile /content/my_bigquery_project/models/marts/customers.sql
-- This is a refactored version of your original complex query.
-- It now builds from clean staging models instead of raw sources.

with
orders as (
    select * from {{ ref('stg_orders') }}
),

customers as (
    select * from {{ ref('stg_customers') }}
),

payments as (
    select * from {{ ref('stg_payments') }}
),

paid_orders as (
    select
        orders.order_id,
        orders.customer_id,
        orders.order_date as order_placed_at,
        orders.order_status,
        -- Summing payments for each order
        sum(case when payments.payment_status = 'success' then payments.amount end) as total_amount_paid,
        max(case when payments.payment_status = 'success' then payments.created_at end) as payment_finalized_date
    from orders
    left join payments on orders.order_id = payments.order_id
    group by 1, 2, 3, 4
),

customer_orders as (
    select
        customer_id,
        min(order_date) as first_order_date,
        max(order_date) as most_recent_order_date,
        count(order_id) as number_of_orders
    from orders
    group by 1
)

select
    p.order_id,
    p.customer_id,
    c.full_name as customer_name,
    p.order_placed_at,
    p.total_amount_paid,
    case
        when c_orders.first_order_date = p.order_placed_at then 'new'
        else 'return'
    end as customer_type
from paid_orders p
left join customers c on p.customer_id = c.customer_id
left join customer_orders c_orders on p.customer_id = c_orders.customer_id
order by p.order_id

Overwriting /content/my_bigquery_project/models/marts/customers.sql


# Run All Models
Now that you have restructured your project, run dbt run. dbt will automatically figure out the correct dependency order, building your three staging models first and then the final customers mart model.

In [27]:
%cd /content/my_bigquery_project
!dbt run

/content/my_bigquery_project
20:53:00  Running with dbt=1.9.6
20:53:03  Registered adapter: bigquery=1.9.2
20:53:05  Encountered an error:
Compilation Error
  dbt found two sources with the name "jaffle_shop_customers".
  
  Since these resources have the same name, dbt will be unable to find the correct resource
  when looking for source("jaffle_shop", "customers").
  
  To fix this, change the name of one of these resources:
  - source.my_bigquery_project.jaffle_shop.customers (models/staging/sources.yml)
  - source.my_bigquery_project.jaffle_shop.customers (models/sources.yml)


# Generate the Documentation
This command introspects your project and your data warehouse to create the static files for the website.

In [28]:
%cd /content/my_bigquery_project
!dbt docs generate

/content/my_bigquery_project
20:53:09  Running with dbt=1.9.6
20:53:12  Registered adapter: bigquery=1.9.2
20:53:13  Encountered an error:
Compilation Error
  dbt found two sources with the name "jaffle_shop_customers".
  
  Since these resources have the same name, dbt will be unable to find the correct resource
  when looking for source("jaffle_shop", "customers").
  
  To fix this, change the name of one of these resources:
  - source.my_bigquery_project.jaffle_shop.customers (models/staging/sources.yml)
  - source.my_bigquery_project.jaffle_shop.customers (models/sources.yml)


In [29]:
from google.colab import files

# Zip the target directory which contains the documentation website
!zip -r /content/dbt_docs.zip /content/my_bigquery_project/target

print("\n Documentation zipped successfully.")
print("⬇ Click the link below to download 'dbt_docs.zip'.")

# Provide the zipped file for download
files.download('/content/dbt_docs.zip')

  adding: content/my_bigquery_project/target/ (stored 0%)
  adding: content/my_bigquery_project/target/run/ (stored 0%)
  adding: content/my_bigquery_project/target/run/my_bigquery_project/ (stored 0%)
  adding: content/my_bigquery_project/target/run/my_bigquery_project/models/ (stored 0%)
  adding: content/my_bigquery_project/target/run/my_bigquery_project/models/marts/ (stored 0%)
  adding: content/my_bigquery_project/target/run/my_bigquery_project/models/marts/customers.sql (deflated 68%)
  adding: content/my_bigquery_project/target/run/my_bigquery_project/models/marts/customers_legacy.sql (deflated 68%)
  adding: content/my_bigquery_project/target/partial_parse.msgpack (deflated 85%)
  adding: content/my_bigquery_project/target/manifest.json (deflated 88%)
  adding: content/my_bigquery_project/target/run_results.json (deflated 58%)
  adding: content/my_bigquery_project/target/compiled/ (stored 0%)
  adding: content/my_bigquery_project/target/compiled/my_bigquery_project/ (stored 0%

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Refactor the customers Model
Rationale: The initial customers model, while functional, was complex and difficult to read. This phase refactors that single, monolithic query into a highly structured and optimized model. We will apply several best practices:

- Code Style: We'll format the SQL to be more readable with consistent casing and line breaks.
- CTE Structure: We'll organize the query into a standard, logical flow: import raw data first, then perform sequential logical transformations, and end with a simple final SELECT statement.
- Performance Tuning: We'll replace expensive self-joins with more performant and readable SQL window functions.
- This process transforms the model from a simple script into a piece of analytics engineering that is easy to debug, extend, and understand.

In [30]:
%%writefile /content/my_bigquery_project/models/marts/customers.sql

with
-- Import CTEs
-- Import staging models into the dbt model
customers as (
    select * from {{ ref('stg_customers') }}
),

orders as (
    select * from {{ ref('stg_orders') }}
),

payments as (
    select * from {{ ref('stg_payments') }}
),

-- Logical CTEs
-- Create an aggregated payments table
aggregated_payments as (
    select
        order_id,
        sum(case when payment_status = 'success' then amount end) as total_amount_paid
    from payments
    group by 1
),

-- Combine orders and payments to create a final orders table
final_orders as (
    select
        orders.order_id,
        orders.customer_id,
        orders.order_date as order_placed_at,
        orders.order_status,
        coalesce(aggregated_payments.total_amount_paid, 0) as total_amount_paid
    from orders
    left join aggregated_payments on orders.order_id = aggregated_payments.order_id
),

-- Final CTE
-- Join customers to orders and add window functions
final as (
    select
        final_orders.order_id,
        final_orders.customer_id,
        final_orders.order_placed_at,
        customers.full_name as customer_name,
        final_orders.total_amount_paid,

        -- Calculate customer lifetime value
        sum(final_orders.total_amount_paid) over (
            partition by final_orders.customer_id
            order by final_orders.order_placed_at
        ) as customer_lifetime_value,

        -- Identify the customer's first order
        rank() over (
            partition by final_orders.customer_id
            order by final_orders.order_placed_at
        ) = 1 as is_first_order

    from final_orders
    left join customers on final_orders.customer_id = customers.customer_id
)

-- Simple select statement
select * from final

Overwriting /content/my_bigquery_project/models/marts/customers.sql


In [31]:
%cd /content/my_bigquery_project
!dbt run --select customers

/content/my_bigquery_project
20:53:18  Running with dbt=1.9.6
20:53:21  Registered adapter: bigquery=1.9.2
20:53:23  Encountered an error:
Compilation Error
  dbt found two sources with the name "jaffle_shop_customers".
  
  Since these resources have the same name, dbt will be unable to find the correct resource
  when looking for source("jaffle_shop", "customers").
  
  To fix this, change the name of one of these resources:
  - source.my_bigquery_project.jaffle_shop.customers (models/staging/sources.yml)
  - source.my_bigquery_project.jaffle_shop.customers (models/sources.yml)


# Preview the refactored output

In [32]:
final_model_dataset = 'jaffle_shop_dbt'
final_model_name = 'customers'

model_table_id = f"`{project_id}.{final_model_dataset}.{final_model_name}`"
sql = f"SELECT * FROM {model_table_id} ORDER BY customer_id, order_placed_at LIMIT 20"

print(f"--- Previewing Refactored dbt Model: {final_model_dataset}.{final_model_name} ---\n")

try:
    df_final = pandas_gbq.read_gbq(sql, project_id=project_id)
    display(df_final)
except Exception as e:
    print(f"Could not query the dbt model table. Error: {e}")

--- Previewing Refactored dbt Model: jaffle_shop_dbt.customers ---

Downloading: 100%|██████████|


,order_id,customer_id,order_placed_at,order_status,total_amount_paid,payment_finalized_date,customer_first_name,customer_last_name,transaction_seq,customer_sales_seq,nvsr,customer_lifetime_value,fdos
0,1,1,1/1/2018,returned,10.0,1/1/2018,Michael,P.,1,1,new,10.0,1/1/2018
1,37,1,2/10/2018,completed,23.0,2/10/2018,Michael,P.,37,2,return,33.0,1/1/2018
2,8,2,1/11/2018,returned,23.0,1/11/2018,Shawn,M.,8,1,new,23.0,1/11/2018
3,2,3,1/2/2018,completed,20.0,1/2/2018,Kathleen,P.,2,1,new,20.0,1/2/2018
4,24,3,1/27/2018,completed,26.0,1/27/2018,Kathleen,P.,24,2,return,46.0,1/2/2018
5,69,3,3/11/2018,completed,19.0,3/11/2018,Kathleen,P.,69,3,return,65.0,1/2/2018
6,46,6,2/19/2018,completed,8.0,2/19/2018,Sarah,R.,46,1,new,8.0,2/19/2018
7,10,7,1/14/2018,completed,26.0,1/14/2018,Martin,M.,10,1,new,26.0,1/14/2018
8,28,8,1/29/2018,completed,19.0,1/29/2018,Frank,R.,28,1,new,19.0,1/29/2018
9,70,8,3/12/2018,completed,26.0,3/12/2018,Frank,R.,70,2,return,45.0,1/29/2018


# Phase 7: Building Intermediate and Mart Models
**Rationale:**

With our staging layer in place, we can now build more complex models on top of it. Some transformation logic might be useful for multiple final models (e.g., aggregating payments to the order level). Instead of repeating this logic, we can create an intermediate model. These models are typically materialized as tables for performance and live in a dedicated intermediate folder.

Finally, we create our mart models. These are the final, customer-facing tables that business users or BI tools will query. They join together staging and intermediate models to create well-structured, easy-to-understand datasets, often in the form of fact and dimension tables.

**Create the Final fct_customer_orders.sql Model**

This is our new primary "fact" model. Notice how simple and readable it has become. It only needs to join our int_orders model with our stg_jaffle_shop__customers model and then perform the final business logic calculations using window functions.



In [33]:
%%writefile /content/my_bigquery_project/models/staging/jaffle_shop/stg_jaffle_shop__customers.sql

with source as (

    select * from {{ source('jaffle_shop', 'customers') }}

),

transformed as (

  select
    id as customer_id,
    name as full_name
    -- Note: Your source data had 'name', not 'first_name' and 'last_name'.
    -- We are adapting the model to match your actual data.

  from source

)

select * from transformed

Writing /content/my_bigquery_project/models/staging/jaffle_shop/stg_jaffle_shop__customers.sql


In [34]:
%%writefile /content/my_bigquery_project/models/staging/jaffle_shop/stg_jaffle_shop__orders.sql

with source as (

    select * from {{ source('jaffle_shop', 'orders') }}

),

transformed as (

  select
    id as order_id,
    user_id as customer_id,
    order_date as order_placed_at,
    status as order_status,

    case
        when status not in ('returned','return_pending')
        then order_date
    end as valid_order_date

  from source

)

select * from transformed

Writing /content/my_bigquery_project/models/staging/jaffle_shop/stg_jaffle_shop__orders.sql


In [35]:
%%writefile /content/my_bigquery_project/models/staging/stripe/stg_stripe__payments.sql

with source as (

    -- Note: The instructions had a typo. Your actual source table is 'stripe_payments'.
    select * from {{ source('stripe', 'stripe_payments') }}

),

transformed as (

  select
    id as payment_id,
    orderid as order_id,
    paymentmethod as payment_method,
    status as payment_status,
    -- amount is stored in cents, convert it to dollars
    amount / 100.0 as payment_amount,
    created as payment_created_at

  from source

)

select * from transformed

Writing /content/my_bigquery_project/models/staging/stripe/stg_stripe__payments.sql


# Phase 8: Final Run and Project Documentation
**Rationale:**

With our project fully refactored into a scalable, multi-layered structure, we perform a final dbt run. dbt's Directed Acyclic Graph (DAG) will automatically determine the correct build order: staging views first, then the intermediate table, and finally our fact and dimension mart tables.

A key benefit of a dbt project is its ability to self-document. The dbt docs generate command creates a comprehensive data catalog website, complete with model descriptions, column definitions, and an interactive lineage graph that visualizes the entire data flow.

In [36]:
# Create the intermediate subfolder
!mkdir -p /content/my_bigquery_project/models/marts/intermediate

print("Intermediate directory created.")

Intermediate directory created.


In [37]:
%%writefile /content/my_bigquery_project/models/marts/intermediate/int_orders.sql

with

orders as (

  select * from {{ ref('stg_jaffle_shop__orders') }}

),

payments as (

  select * from {{ ref('stg_stripe__payments') }}

),

completed_payments as (

  select
    order_id,
    max(payment_created_at) as payment_finalized_date,
    sum(payment_amount) as total_amount_paid
  from payments
  where payment_status <> 'fail'
  group by 1

),

paid_orders as (

  select
    orders.order_id,
    orders.customer_id,
    orders.order_placed_at,
    orders.order_status,
    completed_payments.total_amount_paid,
    completed_payments.payment_finalized_date
  from orders
 left join completed_payments on orders.order_id = completed_payments.order_id
)

select * from paid_orders

Writing /content/my_bigquery_project/models/marts/intermediate/int_orders.sql


# Create the Final, Refactored fct_customer_orders.sql Model
This will be our new primary "fact" model. Notice how simple it is now. It just joins our intermediate model (int_orders) with our staging model (stg_jaffle_shop__customers) and performs the final window functions.

In [38]:
%%writefile /content/my_bigquery_project/models/marts/fct_customer_orders.sql

with

customers as (

  select * from {{ ref('stg_jaffle_shop__customers') }}

),

paid_orders as (

  select * from {{ ref('int_orders') }}

),

final as (

  select
    paid_orders.order_id,
    paid_orders.customer_id,
    paid_orders.order_placed_at,
    paid_orders.order_status,
    paid_orders.total_amount_paid,
    paid_orders.payment_finalized_date,
    customers.customer_first_name || ' ' || customers.customer_last_name as customer_name,

    -- sales transaction sequence
    row_number() over (order by paid_orders.order_placed_at, paid_orders.order_id) as transaction_seq,

    -- customer sales sequence
    row_number() over (
        partition by paid_orders.customer_id
        order by paid_orders.order_placed_at, paid_orders.order_id
        ) as customer_sales_seq,

    -- new vs returning customer
    case
      when (
      rank() over (
        partition by paid_orders.customer_id
        order by paid_orders.order_placed_at, paid_orders.order_id
        ) = 1
      ) then 'new'
    else 'return' end as nvsr,

    -- customer lifetime value
    sum(paid_orders.total_amount_paid) over (
      partition by paid_orders.customer_id
      order by paid_orders.order_placed_at, paid_orders.order_id
      ) as customer_lifetime_value,

    -- first day of sale
    first_value(paid_orders.order_placed_at) over (
      partition by paid_orders.customer_id
      order by paid_orders.order_placed_at, paid_orders.order_id
      ) as fdos

  from paid_orders
  left join customers on paid_orders.customer_id = customers.customer_id
)

select * from final

Writing /content/my_bigquery_project/models/marts/fct_customer_orders.sql


In [39]:
# Remove the old, redundant sources.yml file from the models/ directory
!rm /content/my_bigquery_project/models/sources.yml

print("Removed old sources.yml file successfully.")

Removed old sources.yml file successfully.


In [40]:
%%writefile /content/my_bigquery_project/models/marts/customers_legacy.sql
WITH paid_orders as (
    select
        orders.id as order_id,
        orders.user_id as customer_id,
        orders.order_date as order_placed_at,
        orders.status as order_status,
        p.total_amount_paid,
        p.payment_finalized_date,
        c.first_name as customer_first_name,
        c.last_name as customer_last_name
    FROM
        {{ source('jaffle_shop', 'orders') }} as orders
    left join
        (
            select
                orderid as order_id,
                max(created) as payment_finalized_date,
                sum(amount) / 100.0 as total_amount_paid
            from
                -- THIS LINE IS NOW CORRECTED TO POINT TO THE 'stripe' SOURCE
                {{ source('stripe', 'stripe_payments') }}
            where
                status <> 'fail'
            group by
                1
        ) p ON orders.id = p.order_id
    left join
        {{ source('jaffle_shop', 'customers') }} as c on orders.user_id = c.id
),

customer_orders as (
    select
        c.id as customer_id,
        min(order_date) as first_order_date,
        max(order_date) as most_recent_order_date,
        count(orders.id) AS number_of_orders
    from
        {{ source('jaffle_shop', 'customers') }} as c
    left join
        {{ source('jaffle_shop', 'orders') }} as orders on orders.user_id = c.id
    group by
        1
)

select
    p.*,
    ROW_NUMBER() OVER (ORDER BY p.order_id) as transaction_seq,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY p.order_id) as customer_sales_seq,
    CASE
        WHEN c.first_order_date = p.order_placed_at THEN 'new'
        ELSE 'return'
    END as nvsr,
    x.clv_bad as customer_lifetime_value,
    c.first_order_date as fdos
FROM
    paid_orders p
left join
    customer_orders as c USING (customer_id)
LEFT OUTER JOIN
    (
        select
            p.order_id,
            sum(t2.total_amount_paid) as clv_bad
        from
            paid_orders p
        left join
            paid_orders t2 on p.customer_id = t2.customer_id and p.order_id >= t2.order_id
        group by
            1
        order by
            p.order_id
    ) x on x.order_id = p.order_id
ORDER BY
    order_id

Overwriting /content/my_bigquery_project/models/marts/customers_legacy.sql


In [41]:
!rm /content/my_bigquery_project/models/staging/jaffle_shop/stg_customers.sql

print("Removed old staging file.")

Removed old staging file.


In [42]:
%%writefile /content/my_bigquery_project/models/staging/jaffle_shop/stg_jaffle_shop__customers.sql

with source as (

    select * from {{ source('jaffle_shop', 'customers') }}

),

transformed as (

  select
    id as customer_id,
    -- The raw table has FIRST_NAME and LAST_NAME, not a single 'name' column.
    -- We will concatenate them here as per the refactoring instructions.
    FIRST_NAME as customer_first_name,
    LAST_NAME as customer_last_name

  from source

)

select * from transformed

Overwriting /content/my_bigquery_project/models/staging/jaffle_shop/stg_jaffle_shop__customers.sql


In [43]:
%%writefile /content/my_bigquery_project/models/staging/jaffle_shop/stg_jaffle_shop__orders.sql

with source as (

    select * from {{ source('jaffle_shop', 'orders') }}

),

transformed as (

  select
    id as order_id,
    user_id as customer_id,
    order_date as order_placed_at,
    status as order_status

  from source

)

select * from transformed

Overwriting /content/my_bigquery_project/models/staging/jaffle_shop/stg_jaffle_shop__orders.sql


In [44]:
%%writefile /content/my_bigquery_project/models/marts/customers.sql
-- This is a refactored version of your original complex query.
-- It now builds from clean staging models instead of raw sources.

with
orders as (
    -- Corrected ref to point to the new staging model name
    select * from {{ ref('stg_jaffle_shop__orders') }}
),

customers as (
    -- Corrected ref to point to the new staging model name
    select * from {{ ref('stg_jaffle_shop__customers') }}
),

payments as (
    -- Corrected ref to point to the new staging model name
    select * from {{ ref('stg_stripe__payments') }}
),

paid_orders as (
    select
        orders.order_id,
        orders.customer_id,
        orders.order_placed_at,
        orders.order_status,
        -- Summing payments for each order
        sum(case when payments.payment_status = 'success' then payments.payment_amount end) as total_amount_paid,
        max(case when payments.payment_status = 'success' then payments.payment_created_at end) as payment_finalized_date
    from orders
    left join payments on orders.order_id = payments.order_id
    group by 1, 2, 3, 4
),

customer_orders as (
    select
        customer_id,
        min(order_placed_at) as first_order_date,
        max(order_placed_at) as most_recent_order_date,
        count(order_id) as number_of_orders
    from orders
    group by 1
)

select
    p.order_id,
    p.customer_id,
    c.customer_first_name || ' ' || c.customer_last_name as customer_name,
    p.order_placed_at,
    p.total_amount_paid,
    case
        when c_orders.first_order_date = p.order_placed_at then 'new'
        else 'return'
    end as customer_type
from paid_orders p
left join customers c on p.customer_id = c.customer_id
left join customer_orders c_orders on p.customer_id = c_orders.customer_id
order by p.order_id

Overwriting /content/my_bigquery_project/models/marts/customers.sql


# Phase 11: Run and Verify the Refactored Project

In [45]:
%cd /content/my_bigquery_project
!dbt run

/content/my_bigquery_project
20:53:27  Running with dbt=1.9.6
20:53:32  Registered adapter: bigquery=1.9.2
20:53:33  Found 9 models, 1 analysis, 3 sources, 658 macros
20:53:33  
20:53:33  Concurrency: 4 threads (target='dev')
20:53:33  
20:53:33  1 of 9 START sql table model jaffle_shop_dbt.customers_legacy .................. [RUN]
20:53:33  2 of 9 START sql view model jaffle_shop_dbt.stg_jaffle_shop__customers ......... [RUN]
20:53:33  3 of 9 START sql view model jaffle_shop_dbt.stg_jaffle_shop__orders ............ [RUN]
20:53:33  4 of 9 START sql view model jaffle_shop_dbt.stg_orders ......................... [RUN]
20:53:34  2 of 9 OK created sql view model jaffle_shop_dbt.stg_jaffle_shop__customers .... [CREATE VIEW (0 processed) in 0.68s]
20:53:34  5 of 9 START sql view model jaffle_shop_dbt.stg_payments ....................... [RUN]
20:53:34  4 of 9 OK created sql view model jaffle_shop_dbt.stg_orders .................... [CREATE VIEW (0 processed) in 0.77s]
20:53:34  6 of 9 START

# Query the final table to see the results of the fully refactored, production quality dbt project

In [46]:
import pandas as pd
import pandas_gbq

# These variables should still be in memory
# project_id = 'your-project-id-here'
final_model_dataset = 'jaffle_shop_dbt'
final_model_name = 'fct_customer_orders' # The name of our new fact model

model_table_id = f"`{project_id}.{final_model_dataset}.{final_model_name}`"
sql = f"SELECT * FROM {model_table_id} ORDER BY customer_id, order_placed_at LIMIT 20"

print(f"--- Previewing Final Fact Model: {final_model_dataset}.{final_model_name} ---\n")

try:
    df_final = pandas_gbq.read_gbq(sql, project_id=project_id)
    display(df_final)
except Exception as e:
    print(f"Could not query the dbt model table. Error: {e}")

--- Previewing Final Fact Model: jaffle_shop_dbt.fct_customer_orders ---

Downloading: 100%|██████████|


,order_id,customer_id,order_placed_at,order_status,total_amount_paid,payment_finalized_date,customer_name,transaction_seq,customer_sales_seq,nvsr,customer_lifetime_value,fdos
0,1,1,1/1/2018,returned,10.0,1/1/2018,Michael P.,1,1,new,10.0,1/1/2018
1,37,1,2/10/2018,completed,23.0,2/10/2018,Michael P.,31,2,return,33.0,1/1/2018
2,8,2,1/11/2018,returned,23.0,1/11/2018,Shawn M.,2,1,new,23.0,1/11/2018
3,2,3,1/2/2018,completed,20.0,1/2/2018,Kathleen P.,12,1,new,20.0,1/2/2018
4,24,3,1/27/2018,completed,26.0,1/27/2018,Kathleen P.,19,2,return,46.0,1/2/2018
5,69,3,3/11/2018,completed,19.0,3/11/2018,Kathleen P.,62,3,return,65.0,1/2/2018
6,46,6,2/19/2018,completed,8.0,2/19/2018,Sarah R.,40,1,new,8.0,2/19/2018
7,10,7,1/14/2018,completed,26.0,1/14/2018,Martin M.,4,1,new,26.0,1/14/2018
8,28,8,1/29/2018,completed,19.0,1/29/2018,Frank R.,23,1,new,19.0,1/29/2018
9,70,8,3/12/2018,completed,26.0,3/12/2018,Frank R.,63,2,return,45.0,1/29/2018
